In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import scipy.io
import json

from tqdm import tqdm
from utilities import create_file_list


In [2]:
def imshow(image):
    # print(f'Shape = {image.shape}, min value = {np.min(image)}, max value = {np.max(image)}')
    image = np.reshape(image, (image.shape[0], image.shape[1], -1))
    plt.figure(figsize=(20, 10))
    if image.shape[2] == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        plt.imshow(image, vmin=0)
    else:
        plt.imshow(image, cmap='gray', vmin=0)
    plt.show()
    
def constraint_range_01(image):
    image = (image - np.min(image))/(np.max(image) - np.min(image))
    return image

def constraint_min_0(image):
    image = (image - np.min(image))
    return image

def constraint_cut_0(image):
    image[image < 0] = 0
    return image

In [3]:
def add_rain(city_name, image_name, rain_level, rain_pattern, alpha=0.01, beta=0.005, a=0.01, show_disparity=False, show_depth_raw=False, show_depth=False, show_pattern=False, show_rain_intensity=False, show_rain=False, show_fog=False, show_original=False, show_output=True, show_sample=False):    
    city_image_name = city_name + '/' + image_name
    camera_calibration_path = 'data/Cityscapes/camera_trainvaltest/camera/train/' + city_image_name + '_camera.json'
    camera_calibration_file = open(camera_calibration_path)
    camera_calibration = json.load(camera_calibration_file)
    camera_calibration_file.close()
    baseline = camera_calibration['extrinsic']['baseline']
    fxy = (camera_calibration['intrinsic']['fx'] + camera_calibration['intrinsic']['fy'])/2

    image_path = 'data/Cityscapes/leftImg8bit_trainvaltest/leftImg8bit/train/' + city_image_name + '_leftImg8bit.png'
    image = cv2.imread(image_path)
    
    rain_pattern_path = 'rain-rendering/data/output/cityscapes/leftImg8bit/train/' + city_name + '/rain/' + rain_level + '/rain_mask/' + rain_pattern + '_leftImg8bit.png'
    R_pattern = cv2.imread(rain_pattern_path)
    R_pattern = cv2.resize(R_pattern, (2048, 1024))
    R_pattern = np.average(R_pattern, axis=2)
    R_pattern = constraint_min_0(R_pattern)/255
    # R_pattern = constraint_range_01(R_pattern)
    R_pattern = R_pattern.astype(np.float32)
    R_pattern = cv2.ximgproc.guidedFilter(image, R_pattern, 3, 1e-3)
    # R_pattern = constraint_range_01(R_pattern)
    R_pattern = constraint_cut_0(R_pattern)

    disparity_path = 'data/Cityscapes/disparity_trainvaltest/disparity/train/' + city_image_name + '_disparity.png'
    disparity = cv2.imread(disparity_path, cv2.IMREAD_UNCHANGED).astype(np.float32)
    disparity[disparity > 0] = (disparity[disparity > 0] - 1)/256
    if show_disparity:
        print('Disparity')
        imshow(disparity)

#     depth = baseline*fxy/disparity
#     # depth[depth == np.Inf] = 0
#     depth[depth == np.Inf] = np.average(depth[depth != np.Inf])
    
#     if show_depth_raw:
#         print('Raw depth map')
#         imshow(depth)
    
#     depth_inverted_rain_path = 'data/RainCityscapes/leftImg8bit_trainval_rain/depth_rain/train/' + city_image_name + '_000019_depth_rain.png'
#     depth_inverted_rain = cv2.imread(depth_inverted_rain_path).astype(np.float32)
#     depth_rain = 255 - np.average(depth_inverted_rain, axis=2)
    
    depth_refined_path = 'data/Foggy Cityscapes/depth_stereoscopic_trainvaltest/depth_stereoscopic/train/' + city_image_name + '_depth_stereoscopic.mat'
    depth_refined = scipy.io.loadmat(depth_refined_path)
    depth_refined = depth_refined['depth_map'].astype(np.float32)
    
    depth = depth_refined

    A0 = 1.0
    d1 = 2*fxy*a

    # (4)
    A = 1 - np.exp(-beta*depth)
    A_guided = cv2.ximgproc.guidedFilter(image, A, 10, 1e-3)
    A_guided = constraint_cut_0(A_guided)
    A_guided = constraint_range_01(A_guided)
    
    # (3)
    t_r = np.exp(-alpha*np.maximum(d1, depth))
    t_r_guided = cv2.ximgproc.guidedFilter(image, t_r, 10, 1e-3)
    t_r_guided = constraint_cut_0(t_r_guided)
    t_r_guided = constraint_range_01(t_r_guided)

    # (2)
    R = np.multiply(R_pattern, t_r_guided)
    
    if show_depth:
        print('Refined depth map')
        imshow(depth)
        imshow(np.log(depth))
        
    if show_rain_intensity:
        print('Rain streaks intensity')
        # imshow(t_r)
        imshow(t_r_guided)
    
    if show_pattern:
        print('Rain pattern')
        imshow(R_pattern)

    if show_rain:
        print('Rain layer')
        imshow(R)
        
    if show_fog and show_depth:
        print('Depth map')
        imshow(depth)
        
    if show_fog:
        print('Fog layer')
        # imshow(A)
        imshow(A_guided)

    # (1)
    output = np.zeros_like(image)
    for c in range(output.shape[2]):
        output[:, :, c] = image[:, :, c]*(1 - R - A_guided) + 255*(R + A0*A_guided)

    if show_original:
        print('Original')
        imshow(image)
        
    if show_output:
        print('Output')
        imshow(output)
    
    if show_sample:
        output_sample_path = 'data/RainCityscapes/leftImg8bit_trainval_rain/leftImg8bit_rain/train/' + image_name + '_000019_leftImg8bit_rain_alpha_0.01_beta_0.005_dropsize_0.01_pattern_6.png'
        output_sample = cv2.imread(output_sample_path)
        print('Sample')
        imshow(output_sample)
        
    return output

In [4]:
data_cityscapes_dir = '/home/tunx404/Miscellaneous/18744/data/Cityscapes/leftImg8bit_trainvaltest/leftImg8bit/train'
city_name_list, city_path_list = create_file_list(data_cityscapes_dir)
print(city_name_list)

# %cd data/output_rain
# for city_name in city_name_list:
#     !mkdir $city_name

['aachen', 'bochum', 'bremen', 'cologne', 'darmstadt', 'dusseldorf', 'erfurt', 'hamburg', 'hanover', 'jena', 'krefeld', 'monchengladbach', 'strasbourg', 'stuttgart', 'tubingen', 'ulm', 'weimar', 'zurich']


In [6]:
# for c in range(2, len(city_name_list)):
for c in [2]:
    city_name = city_name_list[c]
    city_path = city_path_list[c]
    
    print(c)
    print(city_name)

    image_name_list, _ = create_file_list(city_path)
    for i in range(len(image_name_list)):
        temp = image_name_list[i].split('_')
        image_name_list[i] = temp[0] + '_' + temp[1] + '_' + temp[2]
    print(image_name_list)

    output_dir = 'data/output_rain/'
    for i in tqdm(range(len(image_name_list))):
        city_name  = city_name
        image_name = image_name_list[i]

        rain_pattern = image_name
        type_list = ['H', 'M', 'S']
        rain_level_list = ['20mm', '5mm', '2mm']
        alpha_list = [1e-2, 2e-3, 1e-4]
        beta_list  = [1e-2, 2e-3, 1e-4]
        a = 0.1

        for (type, rain_level, alpha, beta) in list(zip(type_list, rain_level_list, alpha_list, beta_list)):
            output_file_name = output_dir + city_name + '/' + image_name + '_' + type + '.png'
            # print(output_file_name)
            output = add_rain(city_name, image_name, rain_level, rain_pattern, alpha, beta, a, show_output=False)
            cv2.imwrite(output_file_name, output)

14
tubingen
['tubingen_000000_000019', 'tubingen_000001_000019', 'tubingen_000002_000019', 'tubingen_000003_000019', 'tubingen_000004_000019', 'tubingen_000005_000019', 'tubingen_000006_000019', 'tubingen_000007_000019', 'tubingen_000008_000019', 'tubingen_000009_000019', 'tubingen_000010_000019', 'tubingen_000011_000019', 'tubingen_000012_000019', 'tubingen_000013_000019', 'tubingen_000014_000019', 'tubingen_000015_000019', 'tubingen_000016_000019', 'tubingen_000017_000019', 'tubingen_000018_000019', 'tubingen_000019_000019', 'tubingen_000020_000019', 'tubingen_000021_000019', 'tubingen_000022_000019', 'tubingen_000023_000019', 'tubingen_000024_000019', 'tubingen_000025_000019', 'tubingen_000026_000019', 'tubingen_000027_000019', 'tubingen_000028_000019', 'tubingen_000029_000019', 'tubingen_000030_000019', 'tubingen_000031_000019', 'tubingen_000032_000019', 'tubingen_000033_000019', 'tubingen_000034_000019', 'tubingen_000035_000019', 'tubingen_000036_000019', 'tubingen_000037_000019',

100%|███████████████████████████████████████████████████████| 144/144 [04:16<00:00,  1.78s/it]


15
ulm
['ulm_000000_000019', 'ulm_000001_000019', 'ulm_000002_000019', 'ulm_000003_000019', 'ulm_000004_000019', 'ulm_000005_000019', 'ulm_000006_000019', 'ulm_000007_000019', 'ulm_000008_000019', 'ulm_000009_000019', 'ulm_000010_000019', 'ulm_000011_000019', 'ulm_000012_000019', 'ulm_000013_000019', 'ulm_000014_000019', 'ulm_000015_000019', 'ulm_000016_000019', 'ulm_000017_000019', 'ulm_000018_000019', 'ulm_000019_000019', 'ulm_000020_000019', 'ulm_000021_000019', 'ulm_000022_000019', 'ulm_000023_000019', 'ulm_000024_000019', 'ulm_000025_000019', 'ulm_000026_000019', 'ulm_000027_000019', 'ulm_000028_000019', 'ulm_000029_000019', 'ulm_000030_000019', 'ulm_000031_000019', 'ulm_000032_000019', 'ulm_000033_000019', 'ulm_000034_000019', 'ulm_000035_000019', 'ulm_000036_000019', 'ulm_000037_000019', 'ulm_000038_000019', 'ulm_000039_000019', 'ulm_000040_000019', 'ulm_000041_000019', 'ulm_000042_000019', 'ulm_000043_000019', 'ulm_000044_000019', 'ulm_000045_000019', 'ulm_000046_000019', 'ulm_

100%|█████████████████████████████████████████████████████████| 95/95 [02:50<00:00,  1.80s/it]


16
weimar
['weimar_000000_000019', 'weimar_000001_000019', 'weimar_000002_000019', 'weimar_000003_000019', 'weimar_000004_000019', 'weimar_000005_000019', 'weimar_000006_000019', 'weimar_000007_000019', 'weimar_000008_000019', 'weimar_000009_000019', 'weimar_000010_000019', 'weimar_000011_000019', 'weimar_000012_000019', 'weimar_000013_000019', 'weimar_000014_000019', 'weimar_000015_000019', 'weimar_000016_000019', 'weimar_000017_000019', 'weimar_000018_000019', 'weimar_000019_000019', 'weimar_000020_000019', 'weimar_000021_000019', 'weimar_000022_000019', 'weimar_000023_000019', 'weimar_000024_000019', 'weimar_000025_000019', 'weimar_000026_000019', 'weimar_000027_000019', 'weimar_000028_000019', 'weimar_000029_000019', 'weimar_000030_000019', 'weimar_000031_000019', 'weimar_000032_000019', 'weimar_000033_000019', 'weimar_000034_000019', 'weimar_000035_000019', 'weimar_000036_000019', 'weimar_000037_000019', 'weimar_000038_000019', 'weimar_000039_000019', 'weimar_000040_000019', 'weim

100%|███████████████████████████████████████████████████████| 142/142 [04:05<00:00,  1.73s/it]


17
zurich
['zurich_000000_000019', 'zurich_000001_000019', 'zurich_000002_000019', 'zurich_000003_000019', 'zurich_000004_000019', 'zurich_000005_000019', 'zurich_000006_000019', 'zurich_000007_000019', 'zurich_000008_000019', 'zurich_000009_000019', 'zurich_000010_000019', 'zurich_000011_000019', 'zurich_000012_000019', 'zurich_000013_000019', 'zurich_000014_000019', 'zurich_000015_000019', 'zurich_000016_000019', 'zurich_000017_000019', 'zurich_000018_000019', 'zurich_000019_000019', 'zurich_000020_000019', 'zurich_000021_000019', 'zurich_000022_000019', 'zurich_000023_000019', 'zurich_000024_000019', 'zurich_000025_000019', 'zurich_000026_000019', 'zurich_000027_000019', 'zurich_000028_000019', 'zurich_000029_000019', 'zurich_000030_000019', 'zurich_000031_000019', 'zurich_000032_000019', 'zurich_000033_000019', 'zurich_000034_000019', 'zurich_000035_000019', 'zurich_000036_000019', 'zurich_000037_000019', 'zurich_000038_000019', 'zurich_000039_000019', 'zurich_000040_000019', 'zuri

100%|███████████████████████████████████████████████████████| 122/122 [03:34<00:00,  1.76s/it]


<!-- # Coefficient: alpha -->